# Part 3: Policy Insights

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/eu_sdg_performance_data.csv")

tier_labels = {0: "Needs Improvement", 1: "Good Performers", 2: "Leaders"}
df["tier_label"] = df["performance_tier"].map(tier_labels)

# Identify feature columns
drop_cols = {"Country", "year", "performance_tier", "sdgi_score", "tier_label"}
feature_cols = [c for c in df.columns if c not in drop_cols]

In [3]:
# Median values per tier
medians_by_tier = df.groupby("tier_label")[feature_cols].median()

# Compute gaps between tiers
gap_NI_to_GP = (
    medians_by_tier.loc["Good Performers"] - 
    medians_by_tier.loc["Needs Improvement"]
).sort_values(ascending=False)

gap_GP_to_L = (
    medians_by_tier.loc["Leaders"] - 
    medians_by_tier.loc["Good Performers"]
).sort_values(ascending=False)

# Top-3 improvement areas
print("=== Top-3 Improvement Areas (Needs Improvement → Good Performers) ===")
print(gap_NI_to_GP.head(3))

print("\n=== Top-3 Improvement Areas (Good Performers → Leaders) ===")
print(gap_GP_to_L.head(3))

=== Top-3 Improvement Areas (Needs Improvement → Good Performers) ===
sdg8_income     6388.0
sdg9_digital      21.0
sdg16_cpi         20.0
dtype: float64

=== Top-3 Improvement Areas (Good Performers → Leaders) ===
sdg8_income     5146.0
sdg7_eurenew      25.5
sdg16_cpi         23.0
dtype: float64


In [4]:
# Compute Leaders’ thresholds (25th percentile) for targets
leaders_mask = df["performance_tier"] == 2
leaders_thresholds = df.loc[leaders_mask, feature_cols].quantile(0.25)

# Function to compute top-3 gaps for a country-year
def top_gaps_for_row(row, features, thresholds):
    diffs = {}
    for f in features:
        val = row.get(f, np.nan)
        thr = thresholds[f]
        diffs[f] = thr - val  # positive => below Leaders' threshold
    sr = pd.Series(diffs).sort_values(ascending=False)
    top3 = sr.head(3)
    return top3.index.tolist(), top3.values.tolist()

# Apply to underperformers (Needs Improvement countries)
ni_rows = df[df["performance_tier"] == 0].copy()
records = []

for _, row in ni_rows.iterrows():
    keys, vals = top_gaps_for_row(row, feature_cols, leaders_thresholds)
    records.append({
        "country": row.get("country", row.get("Country", "Unknown")),
        "year": int(row.get("year", row.get("Year", -1))),
        "top_gap_1": keys[0], "gap_value_1": vals[0],
        "top_gap_2": keys[1], "gap_value_2": vals[1],
        "top_gap_3": keys[2], "gap_value_3": vals[2],
    })

ni_gaps_df = pd.DataFrame(records)

# Save outputs
ni_gaps_df.to_csv("needs_improvement.csv", index=False)